# Add metadata and weather features to electricity

In [1]:
import sys
import os
sys.path.append("..")
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from webapp.utils.azure_utils import KeyVault, DataLake

In [2]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)
file_system = "energyhub"

In [4]:
# Read data:
metadata = storage.read(file_system, directory = "data_parq/metadata", file_name = "metadata.parq", extension = "parq")
weather = storage.read(file_system, directory = "data_parq/weather", file_name = "weather.parq", extension = "parq")
electricity = storage.read(file_system, directory = "data_parq/meters", file_name = "electricity.parq", extension = "parq")

In [7]:
# Remove unnecessary columns from meta:
metadata = metadata.drop(columns = ['building_id_kaggle', 'site_id_kaggle', 'sq_feet', 'electricity', 'hotwater',
       'chilledwater', 'steam', 'water', 'irrigation', 'solar', 'gas'])

In [10]:
weather = weather.set_index("site_id")
metadata = metadata.set_index("site_id")
df = pd.merge(metadata, weather, on = "site_id", how = "left")

In [11]:
del weather, metadata

In [12]:
df = df.set_index(["building_id", "timestamp"])
electricity = electricity.set_index(["building_id", "timestamp"])

In [13]:
df = pd.merge(electricity, df, left_index = True, right_index = True, how = "left")

In [14]:
del electricity

In [15]:
df = df.rename(columns = {"electricity_x": "electricity"})
df = df.drop(columns = ["electricity_y"])

KeyError: "['electricity_y'] not found in axis"

In [ ]:
df = df.reset_index()